In [18]:
import numba
from numba import types, typed
import numpy as np
import time
import pandas as pd


## Numba Based

In [30]:
spec_node = [('key', types.int32),
             ('mutate_rate', types.float32),
             ('possible_bias_values', types.int64[:]),
             ('bias', types.float64)
            ]

@numba.jitclass(spec_node)
class NodeNB:
    
    def __init__(self, key, mutate_rate, possible_bias_values):
        self.key = key
        self.mutate_rate = mutate_rate
        self.possible_bias_values = possible_bias_values
        self.bias = 0.0
        
    def get_bias(self):
        return self.bias
    
    def get_key(self):
        return self.key
    
    def mutate_bias(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self.bias = np.random.choice(self.possible_bias_values, size=1)[0]

            
@numba.njit()
def get_numba_array(list_):
    return np.array(list_)


@numba.njit(debug=True)
def initialize_nodes_numba(n_nodes, mutate_rate, possible_bias_values):
    nodes = {}
    for i in range(n_nodes):
        node = NodeNB(i, 
                      mutate_rate, 
                      possible_bias_values)
        node.mutate_bias()
        nodes[i] = node
    return nodes

In [21]:
mutate_rate = 0.5
possible_bias_values = list(range(10))
typed_possible_bias_values = get_numba_array(possible_bias_values)

In [22]:
n_nodes = 5000000

In [23]:
%%time
nodes = initialize_nodes_numba(n_nodes, mutate_rate, typed_possible_bias_values)

CPU times: user 1.41 s, sys: 203 ms, total: 1.61 s
Wall time: 1.61 s


In [26]:
def run_experiments(n_nodes_values, n_rep=10):
    def _run_experiments(n_nodes_values, n_rep):
        for n_nodes in n_nodes_values:
            for rep in range(n_rep):
                start = time.time()
                nodes = initialize_nodes_numba(n_nodes, mutate_rate, typed_possible_bias_values)
                elapsed = time.time() - start
                yield n_nodes, elapsed
    experiment_generator = _run_experiments(n_nodes_values, n_rep)
    n_nodes_list, elapsed = zip(*experiment_generator)
    return pd.DataFrame({'n_nodes': n_nodes_list,
                         'elapsed': elapsed})
    

In [29]:
n_nodes_values=[1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]
df = run_experiments(n_nodes_values=n_nodes_values, n_rep=10)
df.to_csv('./numba_accelarated_python.csv', index=False)
df.tail()

,n_nodes,elapsed
75,10000000,3.009400
76,10000000,2.984443
77,10000000,2.977656
78,10000000,2.921214
79,10000000,2.958933
